In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.head()

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label',axis = 1)
y = df['label']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
voc_size = 5000

In [ ]:
df = X.copy()

df.reset_index(inplace = True)

In [ ]:
df.head()

In [ ]:
df['title'][1]

In [ ]:
df['text'][1]

In [ ]:
ps = PorterStemmer()
corpus = []

In [ ]:
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', '', df['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ''.join(review)
    corpus.append(review)

In [ ]:
print(corpus)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [ ]:
sent_length = 20

In [ ]:
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen=sent_length)

embedded_docs

In [ ]:
model = Sequential()

model.add(Embedding(voc_size,40,input_length=sent_length))
model.add(Dropout(0.3))

model.add(LSTM(400))
model.add(Dropout(0.3))

model.add(Dense(1,activation='sigmoid'))
model.add(Dropout(0.5))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

print(model.summary())

In [ ]:
X_final = np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape

In [ ]:
y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.2,random_state=1)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 25,batch_size = 64)

In [ ]:
y_pred = model.predict(X_test)

print(y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(np.round(abs(y_pred)),y_test)

print(acc)

# Bi-Directional LSTMRNN

In [ ]:
model = Sequential()

model.add(Embedding(voc_size,40,input_length=sent_length))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))

model.add(Dense(1,activation='sigmoid'))
model.add(Dropout(0.5))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.2,random_state=1)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 25,batch_size = 64)

In [ ]:
y_pred = model.predict(X_test)

print(y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(np.round(abs(y_pred)),y_test)

print(acc)